In [1]:
'''
@Author: Yitao Qiu
'''
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import math
from copy import deepcopy

### Set date and path

In [2]:
work_path = "C:\Save"
# Current support product: btc, eth
product = 'eth'
trading_date = ['07.03','07.04','07.05','07.06','07.07','07.08','07.09','07.10','07.11','07.12','07.13','07.14']

### Read data

In [3]:
appended_data = []
for i in range(len(trading_date)):
    file_path = os.path.join(work_path, f'books_{product}_2022.{trading_date[i]}.csv')
    data = pd.read_csv(file_path)
    appended_data.append(data)
appended_data = pd.concat(appended_data)
appended_data = appended_data.drop(columns=['sym'])
appended_data.rename(columns={'time':'date'}, inplace=True)
appended_data

,date,bid1,ask1,bid2,ask2,bid3,ask3,bid4,ask4,bid5,...,bidVol6,askVol6,bidVol7,askVol7,bidVol8,askVol8,bidVol9,askVol9,bidVol10,askVol10
0,2022-07-03D00:00:03.216000000,1067.06,1067.07,1067.05,1067.08,1067.02,1067.09,1067.01,1067.11,1067.00,...,5.4173,0.3589,2.9629,4.2417,0.3824,0.1653,3.8782,1.5308,0.0596,5.8650
1,2022-07-03D00:00:03.219000000,1067.06,1067.07,1067.05,1067.08,1067.02,1067.09,1067.01,1067.10,1067.00,...,5.4173,0.3589,2.9629,4.2417,0.3824,0.1653,3.8782,1.5308,0.0596,5.8650
2,2022-07-03D00:00:03.221000000,1067.06,1067.07,1067.05,1067.08,1067.02,1067.09,1067.01,1067.10,1067.00,...,5.4173,0.3589,2.9629,4.2417,0.3824,0.1653,3.8782,1.5308,0.0596,5.8650
3,2022-07-03D00:00:03.222000000,1067.06,1067.07,1067.05,1067.08,1067.02,1067.09,1067.01,1067.10,1067.00,...,5.4173,0.3589,2.9629,4.2417,0.3824,0.1653,3.8782,1.5308,0.0596,5.8650
4,2022-07-03D00:00:03.224000000,1067.06,1067.07,1067.05,1067.08,1067.02,1067.10,1067.01,1067.21,1067.00,...,5.4173,4.2417,2.9629,0.1653,0.3824,1.5308,3.8782,5.8650,0.0596,1.3532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860079,2022-07-14D23:59:59.903000000,1193.41,1193.42,1193.37,1193.43,1193.36,1193.46,1193.35,1193.50,1193.31,...,6.0000,0.5815,13.6307,0.9789,0.3253,16.1640,7.8081,8.4733,11.1619,12.3469
860080,2022-07-15D00:00:00.004000000,1193.41,1193.42,1193.37,1193.43,1193.36,1193.46,1193.35,1193.50,1193.31,...,6.0000,7.4395,5.2450,0.9789,10.0429,16.1640,0.3253,8.4733,0.5248,12.3469
860081,2022-07-15D00:00:00.103000000,1193.41,1193.42,1193.37,1193.43,1193.36,1193.46,1193.35,1193.49,1193.31,...,6.0000,10.9596,5.2450,7.4395,10.0429,0.9789,8.3859,16.1640,0.3253,8.4733
860082,2022-07-15D00:00:00.206000000,1193.41,1193.42,1193.36,1193.43,1193.31,1193.46,1193.29,1193.49,1193.28,...,8.3859,10.9596,0.3253,0.9789,11.1622,11.4050,4.6944,16.1640,0.0461,1.9110


In [4]:
# Formatting the date
appended_data['date'] = pd.to_datetime(appended_data['date'], format='%Y-%m-%dD%H:%M:%S.%f')
# Reset index
appended_data.sort_values(by='date', inplace=True)
appended_data.set_index(keys='date', inplace=True)
appended_data.reset_index(inplace=True)

In [5]:
len(appended_data)

10255344

### Tools for feature extraction

In [6]:
# Calculate the mid-price
appended_data["mid_price"] = (appended_data['ask1'] + appended_data['bid1'])/2

In [ ]:
# Calculate the log mid-price
appended_data["log_mid_price"] = np.log((appended_data['ask1'] + appended_data['bid1'])/2)

In [6]:
# Calculate the bid-ask imbalance
level = [1,2,3,4,5,6,7,8,9,10]
for i in range(len(level)):
    appended_data[f'BAI_{level[i]}'] = (appended_data[f'bidVol{level[i]}'] - appended_data[f'askVol{level[i]}'])/(appended_data[f'bidVol{level[i]}'] + appended_data[f'askVol{level[i]}'])

In [ ]:
# Calculate the culmulative bid-ask imbalance
sum_bid_vol = appended_data[f'bidVol{level[0]}']
for i in range(1,len(level)):
    sum_bid_vol = sum_bid_vol + appended_data[f'bidVol{level[i]}']
sum_ask_vol = appended_data[f'askVol{level[0]}']
for i in range(1,len(level)):
    sum_ask_vol = sum_ask_vol + appended_data[f'askVol{level[i]}']
sum_ask_vol
appended_data[f'BAI_C'] = (sum_bid_vol - sum_ask_vol)/(sum_bid_vol+sum_ask_vol)

In [9]:
appended_data.drop(appended_data.columns[1:41], axis=1, inplace=True)

In [7]:
appended_data

,date,bid1,ask1,bid2,ask2,bid3,ask3,bid4,ask4,bid5,...,askVol6,bidVol7,askVol7,bidVol8,askVol8,bidVol9,askVol9,bidVol10,askVol10,mid_price
0,2022-07-03 00:00:03.216,1067.06,1067.07,1067.05,1067.08,1067.02,1067.09,1067.01,1067.11,1067.00,...,0.3589,2.9629,4.2417,0.3824,0.1653,3.8782,1.5308,0.0596,5.8650,1067.065
1,2022-07-03 00:00:03.219,1067.06,1067.07,1067.05,1067.08,1067.02,1067.09,1067.01,1067.10,1067.00,...,0.3589,2.9629,4.2417,0.3824,0.1653,3.8782,1.5308,0.0596,5.8650,1067.065
2,2022-07-03 00:00:03.221,1067.06,1067.07,1067.05,1067.08,1067.02,1067.09,1067.01,1067.10,1067.00,...,0.3589,2.9629,4.2417,0.3824,0.1653,3.8782,1.5308,0.0596,5.8650,1067.065
3,2022-07-03 00:00:03.222,1067.06,1067.07,1067.05,1067.08,1067.02,1067.09,1067.01,1067.10,1067.00,...,0.3589,2.9629,4.2417,0.3824,0.1653,3.8782,1.5308,0.0596,5.8650,1067.065
4,2022-07-03 00:00:03.224,1067.06,1067.07,1067.05,1067.08,1067.02,1067.10,1067.01,1067.21,1067.00,...,4.2417,2.9629,0.1653,0.3824,1.5308,3.8782,5.8650,0.0596,1.3532,1067.065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10255339,2022-07-14 23:59:59.903,1193.41,1193.42,1193.37,1193.43,1193.36,1193.46,1193.35,1193.50,1193.31,...,0.5815,13.6307,0.9789,0.3253,16.1640,7.8081,8.4733,11.1619,12.3469,1193.415
10255340,2022-07-15 00:00:00.004,1193.41,1193.42,1193.37,1193.43,1193.36,1193.46,1193.35,1193.50,1193.31,...,7.4395,5.2450,0.9789,10.0429,16.1640,0.3253,8.4733,0.5248,12.3469,1193.415
10255341,2022-07-15 00:00:00.103,1193.41,1193.42,1193.37,1193.43,1193.36,1193.46,1193.35,1193.49,1193.31,...,10.9596,5.2450,7.4395,10.0429,0.9789,8.3859,16.1640,0.3253,8.4733,1193.415
10255342,2022-07-15 00:00:00.206,1193.41,1193.42,1193.36,1193.43,1193.31,1193.46,1193.29,1193.49,1193.28,...,10.9596,0.3253,0.9789,11.1622,11.4050,4.6944,16.1640,0.0461,1.9110,1193.415


### Save data

In [ ]:
if not os.path.exists('./datasets'):
    os.makedirs('./datasets')

In [8]:
appended_data.to_csv(f'datasets/{product}_usdt.csv',index=False)

In [11]:
appended_data.to_csv(f'datasets/eth_usdt_feature.csv',index=False)